In [ ]:
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 8.0)
#pylab.rcParams['image.cmap'] = 'rainbow'

from telutil import *

import subprocess
print "Working from Git repository %s" % subprocess.check_output(["git",  "describe", "--long", "--all"])

def timestamp():
    from datetime import datetime
    print "%s %s" % (datetime.now().date(), datetime.now().time())    

def trials(ntrials,nsources,config,name,rmin=0,rmax=200.0,tiono=10,tsky=10000*3600.0,snrthreshold=5.0):
    tp=TelPiercings()

    timestamp()

    for trial in range(ntrials):
        ts=TelSources()
        ts.construct(nsources=nsources, radius=wavelength/stationdiameter)
        tp.construct(ts,config, weight=weight, hiono=hiono)
        if trial==0:
            tp.plot(rmax=HWZ)

        s=numpy.sqrt(tp.assess(nnoll=nnoll, rmax=HWZ, doplot=(trial==0)))
        print "Trial %d, max(s)=%.1f" % (trial, numpy.max(s))

        if trial==0:
            Save=s/float(ntrials)
        else:
            Save=Save+s/float(ntrials)
    J=len(Save[Save>snrthreshold])
    if J<1:
        J=1
    DR=TelIono().dr(J,tsky=tsky,B=2*HWZ)
    itsky=TelIono().tsky(J,DR=1e5,B=2*HWZ)
    stdphase=numpy.sqrt(TelIono().varphase(J,B=2*HWZ))

    return {'dr':DR, 's':Save, 'J':J, 'tsky':itsky, 'stdphase':stdphase}

def printstats(stats):
    print "Config, Nsources, peak, J, stdphase, DR, sky, MST" 

    for config in configs:
        for nsources in lsources:
            print "%s, %d, %.1f, %d, %.2f, %.1f, %.1f, %.1f" \
            % (config, nsources, stats[config][nsources]['s'][0], \
               stats[config][nsources]['J'], \
               stats[config][nsources]['stdphase'], \
            10.0*numpy.log10(stats[config][nsources]['dr']), \
               stats[config][nsources]['tsky']/(24.0*3600.0*365.0), 
               mst[config])

random.seed(781490893)

nstations=512
nnoll=1000

stationdiameter=35.0
rcore=0.5
rmin=0.0
hiono=300.0

print "Freq, HWZ, tiono, imagenoise, visnoise, weight, piercethreshold, nsources"


for mfreq in numpy.arange(50.0,210.0,10.0):
    freq=mfreq*1e6
    
    wavelength=300.0/mfreq
    HWZ=hiono*wavelength/stationdiameter # Half width to Zero
    FOV=(0.5*wavelength/stationdiameter)**2

    rhalo=HWZ
    rmax=HWZ

    tiono=10.0*numpy.power(wavelength/3.0,-5.0/6.0)

    # Calculate weight of solution for each pierce point
    imgnoise=sources().tnoise(time=tiono, freq=freq)*(float(nstations)/512)*(35.0/stationdiameter)**2

    visnoise=sources().tnoise(time=tiono, freq=freq)*float(512)*(35.0/stationdiameter)**2

    gainnoise=visnoise/numpy.sqrt(float(nstations))

    piercethreshold=5.0*gainnoise
# Weight is the gain for a 1.0 Jy source"
    weight=1.0/gainnoise**2

    # Number of sources in the beam abouve 5 sigma in tiono
    nsources=FOV*sources().numbers(piercethreshold)
    print "%.1f, %.2f, %.2f, %.4f, %.3f, %.1f, %.3f, %.1f" % (mfreq, HWZ, tiono, imgnoise, visnoise, weight, piercethreshold, nsources)